In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

from ai.TrashClassifier import TrashClassifier
from ai.DataLoader2 import DataLoader
from env import *

EPOCHS = 100
ETA = 1e-2

In [2]:
model = TrashClassifier()
model = model.cuda()

FeatureCNN was loaded.


In [3]:
train_dloader = DataLoader(TRAIN_DATA_PATH, TRASH_CAT)
valid_dloader = DataLoader(VALID_DATA_PATH, TRASH_CAT)

Number of data: 1241
Number of batch: 22
Number of data: 112
Number of batch: 2


In [4]:
optimizer = optim.Adam(model.parameters(), lr=ETA)
criterion = nn.NLLLoss()

In [5]:
top_valid_acc = 0.0

for e in range(EPOCHS):
    
    train_loss = 0.0
    train_acc = 0.0
    valid_loss = 0.0
    valid_acc = 0.0
    
    for x_batch, y_batch in train_dloader.next_batch():
        x_batch = torch.FloatTensor(x_batch).cuda()
        y_batch = torch.LongTensor(y_batch).cuda()
        
        logps = model(x_batch)
        loss = criterion(logps, y_batch)
        
        with torch.no_grad():
            train_loss += loss.item()
            ps = torch.exp(logps)
            
            ps_k, cls_k = ps.topk(1, dim=1)
            equal = cls_k == y_batch.view(*cls_k.size())
            train_acc += torch.mean(equal.type(torch.FloatTensor))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (e+1) % 10 == 0:
        with torch.no_grad():
            model.eval()

            for x_batch, y_batch in valid_dloader.next_batch():
                x_batch = torch.FloatTensor(x_batch).cuda()
                y_batch = torch.LongTensor(y_batch).cuda()

                logps = model(x_batch)
                loss = criterion(logps, y_batch)

                valid_loss += loss.item()

                ps = torch.exp(logps)
                ps_k, cls_k = ps.topk(1, dim=1)
                equal = cls_k == y_batch.view(*cls_k.size())
                valid_acc += torch.mean(equal.type(torch.FloatTensor))

            train_loss /= len(train_dloader)
            train_acc /= len(train_dloader)
            valid_loss /= len(valid_dloader)
            valid_acc /= len(valid_dloader)

            print(f"Epochs: {e+1}/{EPOCHS}")
            print(f"Train loss: {train_loss:.8f}")
            print(f"Train acc: {train_acc:.8f}")
            print(f"Valid loss: {valid_loss:.8f}")
            print(f"Valid acc: {valid_acc:.8f}")
            
            if top_valid_acc < valid_acc:
                top_valid_acc = valid_acc
                model.save(CLF_CKPT_PATH)

            model.train()

Epochs: 10/100
Train loss: 0.19910732
Train acc: 0.93668836
Valid loss: 0.24641160
Valid acc: 0.91071427
Classifier was saved.
Epochs: 20/100
Train loss: 0.16876042
Train acc: 0.95941550
Valid loss: 0.21539031
Valid acc: 0.91071427
Epochs: 30/100
Train loss: 0.11502209
Train acc: 0.96266228
Valid loss: 0.61083136
Valid acc: 0.89285713
Epochs: 40/100
Train loss: 0.09742984
Train acc: 0.97564930
Valid loss: 0.29605058
Valid acc: 0.92857140
Classifier was saved.
Epochs: 50/100
Train loss: 0.07933328
Train acc: 0.98214281
Valid loss: 0.57385683
Valid acc: 0.87500000
Epochs: 60/100
Train loss: 0.14085700
Train acc: 0.98051947
Valid loss: 0.58715311
Valid acc: 0.82142854
Epochs: 70/100
Train loss: 0.05909693
Train acc: 0.97889614
Valid loss: 1.10740915
Valid acc: 0.92857146
Classifier was saved.
Epochs: 80/100
Train loss: 0.10439331
Train acc: 0.97564930
Valid loss: 0.23776975
Valid acc: 0.91071427
Epochs: 90/100
Train loss: 0.07692653
Train acc: 0.98376620
Valid loss: 0.11723721
Valid acc: 